In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from IPython.display import Audio
import librosa
import librosa.display
from tensorflow.contrib import rnn 

# TESTING RNN (TUTORIAL)
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

# prints out full length of array
# np.set_printoptions(threshold=np.nan)

tfrecords_filename = 'D:/Documents/nsynth/nsynth-train.tfrecord'
filter_tfrecords_filename = 'D:/Documents/nsynth/filtered_data_keyboard.tfrecord'

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
# parse TFRecord file into a generator (brian)
# https://github.com/faroit/nsynth-convert
def nsynth_generator(tfrecords_filename):
    for serialized_example in tf.python_io.tf_record_iterator(tfrecords_filename):
        example = tf.train.Example()
        example.ParseFromString(serialized_example)
        f = example.features.feature
        i=0
#         result = tf.train.Example.FromString(serialized_example)
#         print(result)
        data_array = []

        audio = np.array(f['audio'].float_list.value)

        data = {
            'note':
                f['note'].int64_list.value[i],
            'note_str':
                f['note_str'].bytes_list.value[i],
            'instrument':
                f['instrument'].int64_list.value[i],
            'instrument_str':
                f['instrument_str'].bytes_list.value[i],
            'pitch':
                f['pitch'].int64_list.value[i],
            'velocity':
                f['pitch'].int64_list.value[i],
            'samplerate':
                f['sample_rate'].int64_list.value[i],
            'audio':
                np.array(f['audio'].float_list.value),
            'qualities':
                np.array(f['qualities'].int64_list.value),
            'qualities_str':
                np.array(f['qualities_str'].int64_list.value),
            'instrument_family':
                f['instrument_family'].int64_list.value[i],
            'instrument_family_str':
                f['instrument_family_str'].bytes_list.value[i],
            'instrument_source':
                f['instrument_family'].int64_list.value[i],
            'instrument_source_str':
                f['instrument_source_str'].bytes_list.value[i],
        }
#         print(data)

        yield example.SerializeToString(), f, data, audio

In [3]:
# parse TFRecord file into a generator (bryan)
# https://github.com/faroit/nsynth-convert
def nsynth_generator(tfrecords_filename):
    for serialized_example in tf.python_io.tf_record_iterator(tfrecords_filename):
        example = tf.train.Example()
        example.ParseFromString(serialized_example)
        f = example.features.feature
#         result = tf.train.Example.FromString(serialized_example)
#         print(result)

        audio = np.array(f['audio'].float_list.value)

        data = {
            'note':
                f['note'].int64_list.value[0],
            'note_str':
                f['note_str'].bytes_list.value[0],
            'instrument':
                f['instrument'].int64_list.value[0],
            'instrument_str':
                f['instrument_str'].bytes_list.value[0],
            'pitch':
                f['pitch'].int64_list.value[0],
            'velocity':
                f['pitch'].int64_list.value[0],
            'sample_rate':
                f['sample_rate'].int64_list.value[0],
            'audio':
                np.array(f['audio'].float_list.value),
            'qualities':
                np.array(f['qualities'].int64_list.value),
            'qualities_str':
                np.array(f['qualities_str'].bytes_list.value),
            'instrument_family':
                f['instrument_family'].int64_list.value[0],
            'instrument_family_str':
                f['instrument_family_str'].bytes_list.value[0],
            'instrument_source':
                f['instrument_family'].int64_list.value[0],
            'instrument_source_str':
                f['instrument_source_str'].bytes_list.value[0],
        }
#         print(data)

        yield data, audio

In [ ]:
# Filter function to only contain keyboard tag
def filterData(inputFile, outputFile):
    dataset = nsynth_generator(inputFile)
    count = 1
    data = next(dataset)
    writer = tf.python_io.TFRecordWriter(outputFile)
    try:
        while data:
            if data[1]["instrument_family"].int64_list.value[0] == 4:
                writer.write(data[0])
            print(data[2])
            data = next(dataset)
    except StopIteration:
        pass
    writer.close()

In [ ]:
# Testing nsynth_generator function
gen_samples = nsynth_generator(tfrecords_filename)
metadata, audio = next(gen_samples)

In [ ]:
# Filters the 80GB of data to new file total: 51820
outputFile = "filtered_data_keyboard2.tfrecord"
filterData(tfrecords_filename, outputFile)

In [ ]:
# Print out all data
# for example in tf.python_io.tf_record_iterator(tfrecords_filename):
for example in tf.python_io.tf_record_iterator(filter_tfrecords_filename):
    result = tf.train.Example.FromString(example)
    print(result)

In [ ]:
gen_filter = nsynth_generator(filter_tfrecords_filename)

In [ ]:
# Filtered data
# metadata, audio = next(gen_filter)
for i in range(5):
    metadata, audio = next(gen_filter)
print(audio)
plt.figure(figsize=(14, 5))
plt.plot(audio)
Audio(audio, rate=metadata['sample_rate'])

In [ ]:
# print(len(audio))
# print(len(metadata['audio']))
CQT = librosa.amplitude_to_db(librosa.magphase(librosa.cqt(audio, sr=16000))[0], ref=np.max)
# plt.figure(figsize=(14, 5))
# print(CQT)
librosa.display.specshow(CQT, sr=metadata['sample_rate'], x_axis='time', y_axis='hz')
# print(audio)
S = np.abs(librosa.stft(audio))
# print(librosa.stft(audio))
# print(S)
comps, acts = librosa.decompose.decompose(S)
print(len(comps))
print(len(comps[0]))
print(len(acts))
print(len(acts[0]))
# print(comps)
# print(acts)

In [4]:
# RNN code
# source: https://pythonprogramming.net/rnn-tensorflow-python-machine-learning-tutorial/
hm_epochs = 3
n_classes = 8
batch_size = 1025

# chunk_size = 25
# n_chunks = 41
chunk_size = 4
n_chunks = 2
rnn_size = 128

# hm_epochs = 3
# n_classes = 10
# batch_size = 128
# chunk_size = 28
# n_chunks = 28
# rnn_size = 128

In [5]:
x = tf.placeholder('float', [None, n_chunks, chunk_size])
y = tf.placeholder('float', [None,None])
print(x.get_shape())

(?, 2, 4)


In [6]:
def recurrent_neural_network(x):
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size,n_classes])),
             'biases':tf.Variable(tf.random_normal([n_classes]))}
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, 8])
    x = tf.split(x, n_chunks, 0)
    print(x)
    lstm_cell = rnn.BasicLSTMCell(rnn_size,state_is_tuple=True) 
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']

    return output

In [ ]:
# REFERENCE CODE (dont run this code)
def train_neural_network(x):
#     tf.reset_default_graph()
    prediction = recurrent_neural_network(x)
    # OLD VERSION:
    #cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    # NEW:
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    

    with tf.Session() as sess:
        # OLD:
        #sess.run(tf.initialize_all_variables())
        # NEW:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
#             for metadata, audio in gen_filter:
                ## need to change batch to audio
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels}))
        sess.close()

train_neural_network(x)

In [ ]:
int(mnist.train.num_examples)

In [7]:
prediction = recurrent_neural_network(x)
cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=y) )
optimizer = tf.train.AdamOptimizer().minimize(cost)
# MODIFIED CODE
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    cnt = 0
    gen_filter = nsynth_generator(filter_tfrecords_filename)
    metadata, audio = next(gen_filter)

    for epoch in range(hm_epochs):
        epoch_loss = 0   
        cnt=0
#         try:
#             while metadata:
#                 if(cnt == 25): break
                
#                 X = librosa.cqt(audio)
#                 Xdb = librosa.amplitude_to_db(abs(X))
#                 print(len(X))
#                 print(len(X[0]))
#                 librosa.display.specshow(Xdb, sr=metadata['sample_rate'], x_axis='time', y_axis='hz')
                
# #                 epoch_x, epoch_y = mnist.train.next_batch(batch_size)           
# #                 epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))
# #                 _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
# #                 epoch_loss += c 
                
#                 metadata, audio = next(gen_filter)
#                 cnt += 1
#         except StopIteration:
#             pass
        for metadata, audio in gen_filter:
            if(cnt == 5): break
            cnt += 1
            S = np.abs(librosa.stft(metadata['audio']))
#             comps, acts = librosa.decompose.decompose(S)
            comps, acts = librosa.decompose.decompose(S, n_components=8)
            print(len(comps), len(comps[0]))
            print(len(acts), len(acts[0]))
            comps = comps.reshape((batch_size, n_chunks, chunk_size))
            print(len(comps), len(comps[0]))
            # STUCK HERE
            _, c = sess.run([optimizer, cost], feed_dict={x: comps, y: acts})
            epoch_loss += c
        print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
        
        # Filter function to only contain keyboard tag
#             print(epoch_y)
            ## need to change batch to audio
#             epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))
#             _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
#             epoch_loss += c

#         print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

#     correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

#     accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
#     print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels}))
    sess.close()


[<tf.Tensor 'split:0' shape=(?, 8) dtype=float32>, <tf.Tensor 'split:1' shape=(?, 8) dtype=float32>]
1025 8
8 126
1025 2


InvalidArgumentError: Number of ways to split should evenly divide the split dimension, but got split_dim 0 (size = 1025) and num_split 2
	 [[Node: split = Split[T=DT_FLOAT, num_split=2, _device="/job:localhost/replica:0/task:0/device:GPU:0"](softmax_cross_entropy_with_logits/concat/axis, Reshape)]]
	 [[Node: Mean/_7 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_424_Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'split', defined at:
  File "C:\Users\imbko\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\imbko\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\imbko\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    ap

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [ ]:
prediction = recurrent_neural_network(x)

In [ ]:
print(prediction)

In [ ]:
comps, acts = librosa.decompose.decompose(S, n_components=8)
print(len(comps))
print(len(comps[1]))
comps = comps.reshape((batch_size, n_chunks, chunk_size))
print(len(comps))
print(len(comps[0]))
print(len(comps[0][0]))

In [ ]:
epoch_x, epoch_y = mnist.train.next_batch(batch_size)
print(len(epoch_x))
print(len(epoch_x[0]))

In [ ]:
# Filter function to only contain keyboard tag
def selectData(inputFile, outputFile):
    dataset = nsynth_generator(inputFile)
    cnt = 0
    arr_cnt = 0
    data = next(dataset)
    writer = tf.python_io.TFRecordWriter(outputFile)
    try:
        while data:
#             if (arr_cnt == 50): break
#             if ~300000 entries -> result ~60000
            if cnt == 5:
                cnt = 0
                writer.write(data[0])
                print(arr_cnt)
                print(data[2])
                arr_cnt+=1
            cnt+=1      
            data = next(dataset)
    except StopIteration:
        pass
    writer.close()

In [ ]:
outputFile = "select_5_interval_data.tfrecord"
selectData(tfrecords_filename, outputFile)